In [ ]:
# ==== KAGGLE SUBMISSION VERSION ==== #
import os
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import pickle
import json
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# Kaggle evaluation - ENABLED FOR SUBMISSION
import kaggle_evaluation.mitsui_inference_server

# ==== Enhanced Configuration for Kaggle Submission ==== #
class Config:
    """Enhanced configuration optimized for Kaggle submission"""
    NUM_TARGET_COLUMNS = 424
    RANDOM_STATE = 42
    CV_FOLDS = 1  # Reduced for faster initialization
    
    # Kaggle-optimized model parameters
    LGBM_PARAMS_KAGGLE = {
        'n_estimators': 15,         # Balanced for speed vs performance
        'learning_rate': 0.3,
        'num_leaves': 24,
        'max_depth': 3,
        'random_state': RANDOM_STATE,
        'verbose': -1,
        'n_jobs': 1,
        'force_row_wise': True,
        'feature_fraction': 0.65,
        'bagging_fraction': 0.65,
        'bagging_freq': 3,
        'min_child_samples': 20,
    }
    
    # Feature engineering parameters
    ROLLING_WINDOWS = [10, 20]
    LAG_PERIODS = [2, 3]
    
    # Adaptive training configuration
    MAX_COMPLEX_MODELS = 10      # Reduced stacking models for Kaggle
    MAX_SIMPLE_MODELS = 30      # Reduced LightGBM for performance
    MIN_SAMPLES_REQUIRED = 100
    
    # Early stopping configuration
    EARLY_STOPPING_ROUNDS = 5       # Reduced for faster training
    VALIDATION_SPLIT = 0.10         # Smaller validation set
    
    @staticmethod
    def get_data_path():
        kaggle_path = Path('/kaggle/input/mitsui-commodity-prediction-challenge')
        local_path = Path("dataset")
        
        if kaggle_path.exists():
            print("🔧 Kaggle environment detected")
            return kaggle_path
        else:
            print("🔧 Local development environment detected")
            return local_path
    
    @staticmethod
    def get_lgbm_params():
        """Get Kaggle-optimized LightGBM parameters"""
        return Config.LGBM_PARAMS_KAGGLE.copy()
    
    @staticmethod
    def is_kaggle_environment():
        """Check if running in Kaggle environment"""
        return Path('/kaggle').exists()

CFG = Config()
data_path = CFG.get_data_path()

# ==== Optimized Feature Engineering Pipeline ==== #
class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Kaggle-optimized feature engineering pipeline"""
    
    def __init__(self, rolling_windows=None, lag_periods=None, enable_heavy_features=True):
        self.rolling_windows = rolling_windows or CFG.ROLLING_WINDOWS
        self.lag_periods = lag_periods or CFG.LAG_PERIODS
        self.enable_heavy_features = enable_heavy_features
        self.fitted_stats_ = {}
        self.feature_names_ = []
        
    def fit(self, X, y=None):
        """Fit feature engineer on training data"""
        X = X.copy()
        feature_cols = [c for c in X.columns if c != 'date_id']
        
        self.feature_names_ = feature_cols
        for col in feature_cols:
            self.fitted_stats_[col] = {
                'mean': X[col].mean(),
                'std': X[col].std(),
                'quantiles': X[col].quantile([0.25, 0.5, 0.75]).to_dict()
            }
        return self
    
    def transform(self, X):
        """Transform data using fitted statistics"""
        if not hasattr(self, 'fitted_stats_'):
            raise ValueError("FeatureEngineer must be fitted before transform")
            
        return self._create_features(X.copy())
    
    def fit_transform(self, X, y=None):
        """Fit and transform in one step"""
        return self.fit(X, y).transform(X)
    
    def _create_features(self, df):
        """Create optimized features for Kaggle submission"""
        feature_count_before = len(df.columns)
        feature_cols = [c for c in df.columns if c != 'date_id']
        
        for col in feature_cols:
            if col == 'date_id':
                continue
                
            try:
                # Essential rolling statistics
                for window in self.rolling_windows:
                    df[f'{col}_rolling_mean_{window}'] = df[col].rolling(window).mean()
                    df[f'{col}_rolling_std_{window}'] = df[col].rolling(window).std()
                    
                # Volatility measures
                df[f'{col}_annual_vol_20'] = df[col].rolling(20).std() * np.sqrt(252)
                df[f'{col}_pct_change'] = df[col].pct_change()
                
                # Lag features
                for lag in self.lag_periods:
                    df[f'{col}_lag_{lag}'] = df[col].shift(lag)
                
                if self.enable_heavy_features:
                    # Higher order statistics
                    df[f'{col}_rolling_skew_10'] = df[col].rolling(10).skew()
                    df[f'{col}_rolling_kurt_10'] = df[col].rolling(10).kurt()
                    
                    # Autocorrelation features
                    df[f'{col}_autocorr_1'] = df[col].rolling(20).apply(
                        lambda x: x.autocorr(lag=1) if len(x.dropna()) > 1 else 0, raw=False
                    )
                    df[f'{col}_autocorr_5'] = df[col].rolling(20).apply(
                        lambda x: x.autocorr(lag=5) if len(x.dropna()) > 5 else 0, raw=False
                    )
                    
                    # Market regime indicators
                    roll_mean = df[col].rolling(10).mean()
                    roll_vol = df[col].rolling(10).std()
                    df[f'{col}_regime_trend_up'] = (roll_mean > roll_mean.shift(1)).astype(int)
                    df[f'{col}_regime_high_vol'] = (roll_vol > roll_vol.quantile(0.75)).astype(int)
                    
                    # Vol-of-vol (volatility clustering)
                    rolling_vol = df[col].rolling(10).std()
                    df[f'{col}_vol_of_vol'] = rolling_vol.rolling(5).std()
                    
            except Exception as e:
                print(f"Warning: Error creating features for {col}: {e}")
                continue

        df = df.fillna(0)
        
        feature_count_after = len(df.columns)
        features_added = feature_count_after - feature_count_before
        print(f"✅ Feature engineering completed: {features_added} features added")
        
        return df

# ==== Optimized Model Management ==== #
class AdaptiveModelManager:
    """Kaggle-optimized model management with faster initialization"""
    
    def __init__(self):
        self.models = {}
        self.feature_columns = {}
        self.model_strategies = {}
        
    def calculate_target_importance(self, train_labels_df):
        """Fast target importance calculation"""
        target_columns = [col for col in train_labels_df.columns if col.startswith('target_')]
        importance_scores = []
        
        for target in target_columns:
            # Quick importance calculation
            variance = train_labels_df[target].var()
            non_null_ratio = train_labels_df[target].notna().sum() / len(train_labels_df)
            combined_importance = variance * non_null_ratio
            importance_scores.append(combined_importance)
            
        return importance_scores
    
    def select_model_strategies(self, target_columns, importance_scores):
        """Optimized model strategy selection for Kaggle"""
        target_scores = list(zip(target_columns, importance_scores))
        target_scores.sort(key=lambda x: x[1], reverse=True)
        
        strategies = {}
        for i, (target, score) in enumerate(target_scores):
            if i < CFG.MAX_COMPLEX_MODELS:
                strategies[target] = 'stacking'
            elif i < CFG.MAX_SIMPLE_MODELS:
                strategies[target] = 'lightgbm'
            else:
                strategies[target] = 'linear'
                
        return strategies
    
    def train_stacking_model(self, X, y, target_name):
        """Fast stacking model training"""
        try:
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=CFG.VALIDATION_SPLIT, random_state=CFG.RANDOM_STATE
            )
            
            lgbm_params = CFG.get_lgbm_params()
            lgbm_model = lgb.LGBMRegressor(**lgbm_params)
            
            estimators = [
                ('lr', Ridge(alpha=1.0, random_state=CFG.RANDOM_STATE)),
                ('lgb', lgbm_model)
            ]
            
            model = StackingRegressor(
                estimators=estimators,
                final_estimator=Ridge(alpha=1.0, random_state=CFG.RANDOM_STATE),
                cv=CFG.CV_FOLDS,
                n_jobs=1
            )
            
            model.fit(X, y)
            self.models[target_name] = model
            self.feature_columns[target_name] = X.columns.tolist()
            
            return model
            
        except Exception as e:
            print(f"❌ Stacking failed for {target_name}: {e}")
            return self.train_linear_model(X, y, target_name)
    
    def train_lightgbm_model(self, X, y, target_name):
        """Optimized LightGBM training for Kaggle"""
        try:
            X_train, X_val, y_train, y_val = train_test_split(
                X, y, test_size=CFG.VALIDATION_SPLIT, random_state=CFG.RANDOM_STATE
            )
            
            lgbm_params = CFG.get_lgbm_params()
            model = lgb.LGBMRegressor(**lgbm_params)
            
            model.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric='rmse',
                callbacks=[
                    lgb.early_stopping(CFG.EARLY_STOPPING_ROUNDS),
                    lgb.log_evaluation(0)
                ]
            )
            
            self.models[target_name] = model
            self.feature_columns[target_name] = X.columns.tolist()
            
            return model
            
        except Exception as e:
            print(f"❌ LightGBM failed for {target_name}: {e}")
            return self.train_linear_model(X, y, target_name)
    
    def train_linear_model(self, X, y, target_name):
        """Fast linear model training"""
        model = Ridge(alpha=1.0, random_state=CFG.RANDOM_STATE)
        model.fit(X, y)
        
        self.models[target_name] = model
        self.feature_columns[target_name] = X.columns.tolist()
        
        return model

def calculate_normalized_correlation_metric(y_true, y_pred):
    """Calculate normalized correlation metric"""
    try:
        base_corr, _ = spearmanr(y_true, y_pred)
        if np.isnan(base_corr):
            return 0.0
            
        residuals = y_true - y_pred
        volatility_factor = np.std(residuals) / (np.std(y_true) + 1e-8)
        normalized_metric = abs(base_corr) / (1 + volatility_factor)
        
        return normalized_metric
        
    except Exception:
        return 0.0

# ==== Stabilization Function ==== #
def _stabilize_and_detie_rows(out_df, date_ids=None):
    """Ensure no flat rows in predictions"""
    out_df = out_df.astype(np.float32)
    out_df[:] = np.nan_to_num(out_df.values, nan=0.0, posinf=0.0, neginf=0.0)
    n_rows, n_cols = out_df.shape
    
    if date_ids is None:
        date_ids = np.zeros(n_rows, dtype=int)
        
    vals = out_df.to_numpy(np.float32)
    row_stds = np.std(vals, axis=1)
    flat_mask = row_stds < 1e-15
    
    if np.any(flat_mask):
        for r_idx in np.where(flat_mask)[0]:
            rng = np.random.default_rng(int(date_ids[r_idx]) + 131071)
            noise = rng.normal(loc=0.0, scale=1.0, size=n_cols).astype(np.float32)
            scale = (1.0 + abs(float(np.mean(vals[r_idx])))) * 1e-6
            vals[r_idx] = vals[r_idx] + noise * scale
        out_df.iloc[:, :] = vals
        
    return out_df

# ==== Kaggle-Optimized Pipeline Predictor ==== #
class PipelinePredictor:
    """Kaggle-optimized prediction pipeline"""
    
    def __init__(self):
        self.feature_pipeline = None
        self.model_manager = AdaptiveModelManager()
        self.is_fitted = False
        self.fallback_model = None
        self.original_data = None
    
    def fit(self, train_df, train_labels_df):
        """Fast fitting for Kaggle submission with 15-minute constraint"""
        print("🚀 Fitting Kaggle submission pipeline...")
        
        self.original_data = train_df.copy()
        
        # Always use Kaggle-optimized features
        rolling_windows = CFG.ROLLING_WINDOWS
        lag_periods = CFG.LAG_PERIODS
        enable_heavy = True
        print("🔧 Using Kaggle-optimized features")
        
        self.feature_pipeline = FeatureEngineer(
            rolling_windows=rolling_windows,
            lag_periods=lag_periods,
            enable_heavy_features=enable_heavy
        )
        
        X_train = self.feature_pipeline.fit_transform(train_df)
        X_train = X_train.drop(columns=['date_id'])
        
        # Fast fallback model creation
        print("🔧 Training fallback model...")
        self._create_fallback_model(train_df, train_labels_df)
        
        target_columns = [col for col in train_labels_df.columns if col.startswith('target_')]
        importance_scores = self.model_manager.calculate_target_importance(train_labels_df)
        model_strategies = self.model_manager.select_model_strategies(target_columns, importance_scores)
        
        print(f"📋 Training models for {len(target_columns)} targets")
        print(f"   - Stacking models: {sum(1 for s in model_strategies.values() if s == 'stacking')}")
        print(f"   - LightGBM models: {sum(1 for s in model_strategies.values() if s == 'lightgbm')}")
        print(f"   - Linear models: {sum(1 for s in model_strategies.values() if s == 'linear')}")
        
        # Train models with time optimization
        trained_count = 0
        for target in target_columns:
            strategy = model_strategies.get(target, 'linear')
            
            y = train_labels_df[target].dropna()
            common_idx = X_train.index.intersection(y.index)
            X_aligned = X_train.loc[common_idx].fillna(0)
            y_aligned = y.loc[common_idx]
            
            if len(X_aligned) >= CFG.MIN_SAMPLES_REQUIRED:
                try:
                    if strategy == 'stacking':
                        self.model_manager.train_stacking_model(X_aligned, y_aligned, target)
                    elif strategy == 'lightgbm':
                        self.model_manager.train_lightgbm_model(X_aligned, y_aligned, target)
                    else:
                        self.model_manager.train_linear_model(X_aligned, y_aligned, target)
                    trained_count += 1
                except Exception as e:
                    print(f"⚠️ Failed to train {target}: {e}")
        
        print(f"✅ Successfully trained {trained_count} models")
        self.is_fitted = True
        print("✅ Kaggle pipeline fitting completed")
    
    def _create_fallback_model(self, train_df, train_labels_df):
        """Fast fallback model creation"""
        try:
            recent_data = train_df.tail(15).copy()  # Reduced for speed
            feature_cols = [c for c in recent_data.columns if c != 'date_id']
            
            self.fallback_model = {}
            target_columns = [col for col in train_labels_df.columns if col.startswith('target_')]
            
            for target in target_columns[:8]:  # Reduced for speed
                try:
                    recent_targets = train_labels_df[target].tail(15).dropna()
                    if len(recent_targets) >= 3:
                        X_simple = np.arange(len(recent_targets)).reshape(-1, 1)
                        model = LinearRegression()
                        model.fit(X_simple, recent_targets)
                        self.fallback_model[target] = {
                            'model': model,
                            'last_value': recent_targets.iloc[-1],
                            'mean_value': recent_targets.mean()
                        }
                except Exception:
                    continue
                    
            print(f"✅ Created fallback models for {len(self.fallback_model)} targets")
            
        except Exception as e:
            print(f"⚠️ Fallback model creation failed: {e}")
            self.fallback_model = None
        
    def predict(self, test: pl.DataFrame, *label_lags) -> pl.DataFrame:
        """Generate predictions with guaranteed 424 columns"""
        if not self.is_fitted:
            raise ValueError("Pipeline must be fitted before prediction")
            
        test_df = test.to_pandas()
        
        X_test = self.feature_pipeline.transform(test_df)
        X_test = X_test.drop(columns=['date_id'])
        
        # Initialize predictions with exact dimensions
        predictions = np.zeros((len(test_df), CFG.NUM_TARGET_COLUMNS))
        
        for i in range(CFG.NUM_TARGET_COLUMNS):
            target_name = f"target_{i}"
            
            try:
                if target_name in self.model_manager.models:
                    model = self.model_manager.models[target_name]
                    feature_cols = self.model_manager.feature_columns[target_name]
                    X_aligned = X_test[feature_cols]
                    predictions[:, i] = model.predict(X_aligned)
                else:
                    predictions[:, i] = self._generate_fallback_prediction(test_df, target_name)
                    
            except Exception as e:
                predictions[:, i] = self._generate_fallback_prediction(test_df, target_name)
        
        # Create output with exactly 424 columns
        column_names = [f"target_{i}" for i in range(CFG.NUM_TARGET_COLUMNS)]
        out_df = pd.DataFrame(predictions, columns=column_names)
        out_df = _stabilize_and_detie_rows(out_df, test_df.get('date_id'))
        
        # Final verification
        assert out_df.shape[1] == CFG.NUM_TARGET_COLUMNS, f"Expected 424 columns, got {out_df.shape[1]}"
        assert out_df.shape[0] == len(test_df), f"Row count mismatch: {out_df.shape[0]} vs {len(test_df)}"
        
        return pl.DataFrame(out_df)
    
    def _generate_fallback_prediction(self, test_df, target_name):
        """Fast fallback predictions"""
        n_samples = len(test_df)
        
        if self.fallback_model and target_name in self.fallback_model:
            try:
                fallback_info = self.fallback_model[target_name]
                trend_pred = fallback_info['model'].predict([[n_samples]])[0]
                prediction = 0.7 * fallback_info['last_value'] + 0.3 * trend_pred
                return np.full(n_samples, prediction)
            except Exception:
                pass
        
        # Simple fallback
        try:
            feature_cols = [c for c in test_df.columns if c != 'date_id']
            if feature_cols:
                feature_means = test_df[feature_cols].mean(axis=1)
                feature_std = test_df[feature_cols].std(axis=1)
                prediction = feature_means * 0.01 + feature_std * 0.001
                return prediction.values
            else:
                return np.random.normal(0, 0.001, n_samples)
        except Exception:
            return np.random.normal(0, 0.001, n_samples)

# ==== Kaggle Submission Manager ==== #
class KaggleSubmissionManager:
    """Optimized Kaggle submission management"""
    
    def __init__(self):
        self.predictor = None
        self.initialization_attempted = False
        
    def initialize_for_submission(self):
        """Fast initialization for Kaggle submission"""
        if self.initialization_attempted:
            return
            
        try:
            print("🚀 Initializing Kaggle submission pipeline...")
            self.predictor = PipelinePredictor()
            
            train_df = pd.read_csv(data_path / 'train.csv')
            train_labels_df = pd.read_csv(data_path / 'train_labels.csv')
            
            print(f"📊 Loaded data: Features {train_df.shape}, Labels {train_labels_df.shape}")
            
            self.predictor.fit(train_df, train_labels_df)
            
            print("✅ Kaggle submission pipeline initialized successfully")
            
        except Exception as e:
            print(f"❌ Initialization failed: {e}")
            import traceback
            traceback.print_exc()
            self.predictor = self._create_fallback_predictor()
            
        finally:
            self.initialization_attempted = True
    
    def _create_fallback_predictor(self):
        """Minimal fallback predictor with guaranteed 424 columns"""
        class FallbackPredictor:
            def predict(self, test, *args):
                n_samples = len(test)
                fallback_preds = np.random.normal(0, 0.001, (n_samples, CFG.NUM_TARGET_COLUMNS))
                
                column_names = [f"target_{i}" for i in range(CFG.NUM_TARGET_COLUMNS)]
                out_df = pd.DataFrame(fallback_preds, columns=column_names)
                return pl.DataFrame(out_df)
        
        return FallbackPredictor()
    
    def predict(self, test, *label_lags):
        """Main prediction function with verification"""
        if not self.initialization_attempted:
            self.initialize_for_submission()
            
        result = self.predictor.predict(test, *label_lags)
        
        # Final verification for Kaggle submission
        assert result.shape[1] == CFG.NUM_TARGET_COLUMNS, f"Expected 424 columns, got {result.shape[1]}"
        assert result.shape[0] == len(test), f"Row count mismatch: {result.shape[0]} vs {len(test)}"
        
        return result

# Global submission manager
submission_manager = KaggleSubmissionManager()

def predict(test: pl.DataFrame,
           label_lags_1_batch: pl.DataFrame,
           label_lags_2_batch: pl.DataFrame,
           label_lags_3_batch: pl.DataFrame,
           label_lags_4_batch: pl.DataFrame) -> pl.DataFrame:
    """Kaggle submission predict function - guaranteed 424 columns"""
    result = submission_manager.predict(test, label_lags_1_batch, 
                                      label_lags_2_batch, label_lags_3_batch, 
                                      label_lags_4_batch)
    
    # Final safety check
    if result.shape[1] != CFG.NUM_TARGET_COLUMNS:
        print(f"🔧 Final column adjustment: {result.shape[1]} -> {CFG.NUM_TARGET_COLUMNS}")
        # Ensure exact column count
        if result.shape[1] < CFG.NUM_TARGET_COLUMNS:
            missing = CFG.NUM_TARGET_COLUMNS - result.shape[1]
            zeros_df = pl.DataFrame({f"target_{result.shape[1] + i}": [0.0] * len(result) 
                                   for i in range(missing)})
            result = pl.concat([result, zeros_df], how="horizontal")
        else:
            result = result.select([f"target_{i}" for i in range(CFG.NUM_TARGET_COLUMNS)])
    
    print(f"✅ Prediction completed: {result.shape[0]} rows × {result.shape[1]} columns")
    return result

# Kaggle Inference Server - ENABLED FOR SUBMISSION
inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway((str(data_path),))